In [20]:
import measurement
import visu_common
import os
import numpy as np
import scipy

mems = visu_common.get_mems('.')
directions = ['s', 'r']
meas_type = 'latency'

identified_params = {}
for mem in mems:
    for direction in directions:
        # here mem is the folder
        clocks = visu_common.get_clocks_in_folder(
            mem, prefix=f'meas_{direction}_', clock_lambda=lambda m7, m4: m4 >= 60)

        K_list = [] # matrix for the linear regression, 
        y_list = []
        for i, (m7, m4) in enumerate(clocks):
            dir_prefix = os.path.join(mem, f'meas_{direction}_{m7}_{m4}')
            sizes = sorted(visu_common.get_sizes(dir_prefix, size_lambda=lambda size: size<260))
            # row vector of measurements for sizes
            y = measurement.get_and_calc_meas(m4, dir_prefix, sizes, meas_type)
            y_list.append(y[0, :]) # mean and column
            K = np.hstack((1/m7*np.ones((len(sizes), 1)), # const part for sending
                        1/m7*np.array(sizes).reshape(-1, 1), # size dependent part for sending
                        1/m4*np.array(sizes).reshape(-1, 1),
                        1/m4*np.ones((len(sizes), 1))))
            K_list.append(K)
            
        K_matrix = np.vstack(K_list)
        y_vector = np.hstack(y_list) # to column vector

        res = scipy.optimize.lsq_linear(K_matrix, y_vector, bounds=(0, np.inf))
        if mem in identified_params:
            identified_params[mem][direction] = list(res.x)
        else:
            identified_params[mem] = {direction: list(res.x)}

In [21]:
import json

with open('models.json', 'w') as file:
    json.dump(identified_params, file, indent=4)